In [11]:
import os
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from IPython.display import display

In [12]:
COMMODITY = 'magnesium'
METRIC = 'Recall'
ALGO = 'detection_correction'
# ALGO = 'correction'


EDCR_RESULTS_PATH = f'{COMMODITY}/test/predictions/test_{METRIC}_{ALGO}'
OG_RESULTS_PATH = f'{COMMODITY}/test/results_test.csv'

In [13]:
def extract_info_from_filename(filename):
    parts = filename.split("Rule")
    epsilon = parts[0].split()[-1]
    models_base = parts[1].split("for")
    models = models_base[0].strip()
    base = models_base[1].split("_predictions")[0].strip()
    return base, models, epsilon

In [14]:
def calculate_metrics(df):
    y_true = df['True']
    y_pred = df['Predicted']
    accuracy = accuracy_score(y_true, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average=None, labels=[0, 1])
    prior = pd.read_csv(OG_RESULTS_PATH, nrows=1)['Prior'].iloc[0]
    return accuracy, precision, recall, f1, prior

In [15]:
def process_files(path):
    all_results = []  
    for filename in os.listdir(path):
        if filename.startswith('Confident') and filename.endswith("_predictions.csv"):
            base, models, confidence = extract_info_from_filename(filename)
            df = pd.read_csv(os.path.join(path, filename))
            accuracy, precision, recall, f1, prior = calculate_metrics(df)
            row = {
                'Base': base,
                'Models': models,
                'Confidence': confidence,
                'Accuracy': accuracy,
                'Precision (0)': precision[0],
                'Recall (0)': recall[0],
                'F1 (0)': f1[0],
                'Precision (1)': precision[1],
                'Recall (1)': recall[1],
                'F1 (1)': f1[1],
                'Prior': prior  
            }
            all_results.append(row)
    
    # Convert the list of results to a DataFrame
    return pd.DataFrame(all_results)

In [16]:
results_df = process_files(EDCR_RESULTS_PATH)

c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to cont

In [17]:
results_df_one = results_df[results_df['Models'] != 'all']
# pd.set_option('display.max_rows', 200)
results_df_one

,Base,Models,Confidence,Accuracy,Precision (0),Recall (0),F1 (0),Precision (1),Recall (1),F1 (1),Prior
34,CNNA_256_3,CNNA_128_3,0.1,0.196491,0.000000,0.000000,0.000000,0.196491,1.000000,0.328446,0.2
35,CNNA_256_5,CNNA_128_3,0.1,0.235088,0.823529,0.061135,0.113821,0.197761,0.946429,0.327160,0.2
36,CNNA_256_7,CNNA_128_3,0.1,0.196491,0.000000,0.000000,0.000000,0.196491,1.000000,0.328446,0.2
37,CNN_128_3,CNNA_128_3,0.1,0.196491,0.000000,0.000000,0.000000,0.196491,1.000000,0.328446,0.2
38,CNN_128_5,CNNA_128_3,0.1,0.196491,0.000000,0.000000,0.000000,0.196491,1.000000,0.328446,0.2
...,...,...,...,...,...,...,...,...,...,...,...
2323,RNN_64,CNN_32_3,0.9,0.396491,0.800000,0.331878,0.469136,0.194737,0.660714,0.300813,0.2
2324,CNN_256_3,CNN_64_7,0.9,0.389474,0.795699,0.323144,0.459627,0.192708,0.660714,0.298387,0.2
2325,CNN_256_5,CNN_64_7,0.9,0.670175,0.800000,0.786026,0.792952,0.183333,0.196429,0.189655,0.2
2326,CNN_256_7,CNN_64_7,0.9,0.214035,1.000000,0.021834,0.042735,0.200000,1.000000,0.333333,0.2


In [18]:
og_df = pd.read_csv(OG_RESULTS_PATH)
og_df = og_df.drop('Unnamed: 0', axis=1)
og_df = og_df[(og_df['Name'] != 'EDCR')]
pd.set_option('display.max_rows', 400)
og_df

,Name,Params,Accuracy,Precision (0),Recall (0),F1 (0),Precision (1),Recall (1),F1 (1),Prior
0,LSTM,256 layers,0.333333,0.842105,0.209607,0.335664,0.206140,0.839286,0.330986,0.2
1,LSTM,128 layers,0.771930,0.799270,0.956332,0.870775,0.090909,0.017857,0.029851,0.2
2,LSTM,64 layers,0.350877,0.814286,0.248908,0.381271,0.200000,0.767857,0.317343,0.2
3,LSTM,32 layers,0.400000,0.822222,0.323144,0.463950,0.205128,0.714286,0.318725,0.2
4,CNN with Attention,"32 filters, kernel size 7",0.540351,0.769231,0.611354,0.681265,0.135922,0.250000,0.176101,0.2
5,CNN with Attention,"32 filters, kernel size 5",0.196491,0.000000,0.000000,0.000000,0.196491,1.000000,0.328446,0.2
6,CNN with Attention,"32 filters, kernel size 3",0.607018,0.823204,0.650655,0.726829,0.230769,0.428571,0.300000,0.2
7,CNN with Attention,"64 filters, kernel size 7",0.407018,0.806122,0.344978,0.483180,0.197861,0.660714,0.304527,0.2
8,CNN with Attention,"64 filters, kernel size 5",0.803509,0.812274,0.982533,0.889328,0.500000,0.071429,0.125000,0.2
9,CNN with Attention,"64 filters, kernel size 3",0.256140,0.869565,0.087336,0.158730,0.202290,0.946429,0.333333,0.2


### Base Model

Generally for lower confidence (0.1-0.3), the rule is basically all '1'. It's always predicting spikes, so that's why lower confidence results look similar to spikes-dumb-model

Conversely, for higher confidence (0.9-0.95), the rule is becoming more like all '0'. So the higher we go, the less spikes we're predicting, hence making it look more like non-spikes-dumb-model

In [21]:
MODEL_NAME_OF_INTEREST = 'CNNA'
MODEL_PARAM_OF_INTEREST = '64_5'

results_df_all = results_df[(results_df['Base'] == f'{MODEL_NAME_OF_INTEREST}_{MODEL_PARAM_OF_INTEREST}') & (results_df['Models'] == 'all') & (results_df['Base'] != "dumb_spikes") & (results_df['Base'] != "dumb_non_spikes")]
results_df_all

,Base,Models,Confidence,Accuracy,Precision (0),Recall (0),F1 (0),Precision (1),Recall (1),F1 (1),Prior
10,CNNA_64_5,all,0.1,0.196491,0.000000,0.000000,0.000000,0.196491,1.000000,0.328446,0.2
204,CNNA_64_5,all,0.2,0.196491,0.000000,0.000000,0.000000,0.196491,1.000000,0.328446,0.2
398,CNNA_64_5,all,0.3,0.196491,0.000000,0.000000,0.000000,0.196491,1.000000,0.328446,0.2
592,CNNA_64_5,all,0.45,0.196491,0.000000,0.000000,0.000000,0.196491,1.000000,0.328446,0.2
786,CNNA_64_5,all,0.4,0.196491,0.000000,0.000000,0.000000,0.196491,1.000000,0.328446,0.2
980,CNNA_64_5,all,0.55,0.221053,0.888889,0.034934,0.067227,0.199275,0.982143,0.331325,0.2
1174,CNNA_64_5,all,0.5,0.196491,0.000000,0.000000,0.000000,0.196491,1.000000,0.328446,0.2
1368,CNNA_64_5,all,0.6,0.585965,0.817143,0.624454,0.707921,0.218182,0.428571,0.289157,0.2
1562,CNNA_64_5,all,0.7,0.740351,0.806324,0.890830,0.846473,0.218750,0.125000,0.159091,0.2
1756,CNNA_64_5,all,0.8,0.810526,0.818182,0.982533,0.892857,0.600000,0.107143,0.181818,0.2


### Dumb Model (Only Predicts Non-Spikes)

In [20]:
results_df_all = results_df[(results_df['Models'] == 'all') & (results_df['Base'] == "dumb_non_spikes")]
results_df_all

,Base,Models,Confidence,Accuracy,Precision (0),Recall (0),F1 (0),Precision (1),Recall (1),F1 (1),Prior
24,dumb_non_spikes,all,0.1,0.196491,0.000000,0.000000,0.000000,0.196491,1.0,0.328446,0.2
218,dumb_non_spikes,all,0.2,0.196491,0.000000,0.000000,0.000000,0.196491,1.0,0.328446,0.2
412,dumb_non_spikes,all,0.3,0.196491,0.000000,0.000000,0.000000,0.196491,1.0,0.328446,0.2
606,dumb_non_spikes,all,0.45,0.196491,0.000000,0.000000,0.000000,0.196491,1.0,0.328446,0.2
800,dumb_non_spikes,all,0.4,0.196491,0.000000,0.000000,0.000000,0.196491,1.0,0.328446,0.2
994,dumb_non_spikes,all,0.55,0.196491,0.000000,0.000000,0.000000,0.196491,1.0,0.328446,0.2
1188,dumb_non_spikes,all,0.5,0.196491,0.000000,0.000000,0.000000,0.196491,1.0,0.328446,0.2
1382,dumb_non_spikes,all,0.6,0.200000,1.000000,0.004367,0.008696,0.197183,1.0,0.329412,0.2
1576,dumb_non_spikes,all,0.7,0.803509,0.803509,1.000000,0.891051,0.000000,0.0,0.000000,0.2
1770,dumb_non_spikes,all,0.8,0.803509,0.803509,1.000000,0.891051,0.000000,0.0,0.000000,0.2
